# Analyze 

In [7]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import datetime
import json
import folium
import utils
import consts

## Defines

In [8]:
USE_GRID = 'nuts'

In [9]:
if USE_GRID == 'nuts':
    path_timedata = consts.PATH_TIMEDATA_NUTS
    path_grid = consts.PATH_MASTERNUTS
    path_map = consts.PATH_DOCS / 'sunrise-nuts.html'
elif USE_GRID == 'hex':
    path_timedata = consts.PATH_TIMEDATA_HEX
    path_grid = consts.PATH_HEXAGON
    path_map = consts.PATH_DOCS / 'sunrise-hex.html'
else:
    raise ValueError(f"Unknown grid type {USE_GRID}")

## Load data

In [10]:
# Load timedata
time_data = json.load(open(path_timedata, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(path_grid)

# Redice precision
gdf_grid_raw['geometry'] = gdf_grid_raw['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

# Load Countryborders
gdf_borders = gpd.read_file(consts.PATH_BORDERS)
gdf_borders['geometry'] = gdf_borders['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

## Prepare Dataset

In [11]:
# Serialize JSON
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'name': x['name'],
    # 'sunrise_current_dt': x['sunrise']['current']['latest'],
    # 'sunrise_summertime': x['sunrise']['summer']['latest'],
    # 'sunrise_wintertime': x['sunrise']['winter']['latest'],
    'sunrise_current_dt': x['sunrise']['current']['earliest'],
    'sunrise_summertime': x['sunrise']['summer']['earliest'],
    'sunrise_wintertime': x['sunrise']['winter']['earliest'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert current to only time like the others
def dt_to_time(dt):
    dt = datetime.datetime.strptime(dt.split('+')[0], '%Y-%m-%d %H:%M:%S.%f')
    return dt.time()

def time_to_time(dt):
    return datetime.datetime.strptime(dt, '%H:%M:%S.%f').time()

# Convert strings to time
df_timedata['sunrise_current'] = df_timedata['sunrise_current_dt'].apply(dt_to_time)
df_timedata['sunrise_summertime'] = df_timedata['sunrise_summertime'].apply(time_to_time)
df_timedata['sunrise_wintertime'] = df_timedata['sunrise_wintertime'].apply(time_to_time)

# Calculate difference between time_reference and given time
def calculate_difference_to_timereference(dt, time_reference):
    diff = datetime.datetime.combine(datetime.date.min, dt) - datetime.datetime.combine(datetime.date.min, time_reference)
    return round(diff.total_seconds() / 60)

def categorize_time_to_half_hour_numerical(dt):
    h = dt.hour
    if dt.minute < 30:
        return h
    else:
        return h + 0.5
    
def categorize_time_to_half_hour(dt):
    h = dt.hour
    if dt.minute < 30:
        return f"{h:02d}:00 - {h:02d}:30"
    else:
        return f"{h:02d}:30 - {h+1:02d}:00"    

# Current System
df_timedata['sunrise_current_category'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_current_numerical'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Summertime
df_timedata['sunrise_summertime_category'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_summertime_numerical'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Wintertime
df_timedata['sunrise_wintertime_category'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_wintertime_numerical'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

In [12]:
m = folium.Map(
    tiles=None,
    prefer_canvas=False
)

folium.TileLayer(
    'cartodbpositron',
    opacity=0.4
).add_to(m)

# Create bins evenly distributed with 0 in the center
min_value = min(
    gdf_grid['sunrise_current_numerical'].min(),
    gdf_grid['sunrise_summertime_numerical'].min(),
    gdf_grid['sunrise_wintertime_numerical'].min()
)
max_value = max(
    gdf_grid['sunrise_current_numerical'].max(),
    gdf_grid['sunrise_summertime_numerical'].max(),
    gdf_grid['sunrise_wintertime_numerical'].max()
)
bins = [x * 0.5 for x in range(int(min_value * 2), int(max_value * 2) + 1)]

def add_choropleth(name, field, popup_fields):

    cp = folium.Choropleth(
        geo_data=gdf_grid.to_json(default=str),
        name=name,
        data=df_timedata,
        columns=["nuts_id", field],
        key_on="feature.properties.NUTS_ID",
        fill_color="Greys",
        fill_opacity=1,
        line_opacity=0.4,
        bins = bins,
        overlay=True,
        show=(name == "Aktuelles System")
    ).add_to(m)


    folium.GeoJsonTooltip(["name"] + popup_fields).add_to(cp.geojson)    

add_choropleth("Aktuelles System", 'sunrise_current_numerical', ['sunrise_current_category', 'sunrise_current'])
add_choropleth("Sommerzeit", 'sunrise_summertime_numerical', ['sunrise_summertime_category', 'sunrise_summertime'])
add_choropleth("Winterzeit", 'sunrise_wintertime_numerical', ['sunrise_wintertime_category', 'sunrise_wintertime'])

# Add Country borders
boudaries = folium.GeoJson(
    gdf_borders,
    name="Ländergrenzen",
    style_function=lambda x: {
        'color': 'white',
        'weight': 1,
        'fillOpacity': 0,
        'interactive':False
    },
    control=False
).add_to(m)
m.keep_in_front(boudaries)

m.fit_bounds([[34.615, -10.334], [71.114, 34.5896]])

folium.LayerControl(collapsed=False).add_to(m)

utils.addTitle(m, "Frühester Sonnenaufgang")

m.save(path_map)